In [2]:
import random
import re
import numpy as np
import tensorflow as tf
from tensorflow import keras  
from tqdm import tqdm
from transformers import (
    AutoTokenizer,
    TFAutoModelForSequenceClassification,
)

# === CONFIGURATION CONSTANTS ===
LANGUAGE = 'en'
MODEL_TYPE = 'distilbert-base-uncased' if LANGUAGE == 'en' else 'distilbert-base-multilingual-cased'
MODEL_INPUTS = frozenset(['input_ids', 'attention_mask'])
BATCH_SIZE = 16
EVAL_BATCH_SIZE = BATCH_SIZE * 2
EPOCHS = 2       # keep low for demo
MAX_SEQ_LENGTH = 128
NUM_DOCS = 10   # mock dataset size

# === MOCK Avro-Like Data Generator ===
def mock_avro_reader(num_docs=NUM_DOCS):
    samples = [
        ("<p>This agreement is made between the company and the contractor.</p>", True),
        ("<h1>Invoice</h1><p>Date: 2024-09-12</p>", False),
        ("<div>Employment contract for the following individual...</div>", True),
        ("<p>Purchase order for materials...</p>", False),
        ("<div>Lease agreement details: terms, parties, rent.</div>", True),
        ("<p>Meeting notes: Project timeline, deliverables, next steps...</p>", False),
    ]
    for _ in range(num_docs):
        html, is_contract = random.choice(samples)
        yield {
            'body': html,
            'labels': {'contract': is_contract}
        }

# === MOCK html_labels2text ===
def html_labels2text(doc):
    # Very simple HTML-stripping mock
    text = re.sub('<[^<]+?>', '', doc['body'])
    return {'text': text}

# === PREPARE LABELLED DATA ===
labels_with_text = []
avro_reader = mock_avro_reader(NUM_DOCS)
for document in avro_reader:
    document_text = html_labels2text({'body': document['body']})['text']
    label = 1 if document['labels'].get('contract') else 0
    labels_with_text.append((label, document_text))

# === LOAD TOKENIZER ===
tokenizer = AutoTokenizer.from_pretrained(MODEL_TYPE)

def encode(text):
    inputs = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        return_tensors='tf',
        max_length=MAX_SEQ_LENGTH,
        truncation=True,
    )
    return {k: v for k, v in inputs.items() if k in MODEL_INPUTS}

# === ENCODE DATA ===
data = [
    ({k: v[0].numpy() for k, v in encode(text).items()}, label)
    for label, text in tqdm(labels_with_text, desc="Encoding texts")
]

# === TF DATASET CREATION ===
def _data():
    yield from data

dataset = tf.data.Dataset.from_generator(
    _data,
    ({k: tf.int32 for k in MODEL_INPUTS}, tf.int32),
    (
        {k: tf.TensorShape([MAX_SEQ_LENGTH]) for k in MODEL_INPUTS},
        tf.TensorShape([]),
    ),
)
dataset = dataset.shuffle(
    buffer_size=NUM_DOCS, seed=0, reshuffle_each_iteration=False
)

# === SPLIT TRAIN AND TEST ===
total_size = len(data)
train_size = int(0.9 * total_size)
test_size = total_size - train_size
train_dataset = dataset.take(train_size)
test_dataset = dataset.skip(train_size)
train_dataset = train_dataset.batch(BATCH_SIZE).repeat(-1)
test_dataset = test_dataset.batch(EVAL_BATCH_SIZE)

# === LOAD AND COMPILE MODEL ===
model = TFAutoModelForSequenceClassification.from_pretrained(MODEL_TYPE, num_labels=2)
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = ["accuracy"]
model.compile(optimizer=optimizer, loss=loss_fn, metrics=metrics)

steps_per_epoch = train_size // BATCH_SIZE
validation_steps = test_size // EVAL_BATCH_SIZE

# === TRAIN MODEL ===
model.fit(
    train_dataset,
    epochs=EPOCHS,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps
)

# === EVALUATE MODEL ===
results = model.evaluate(test_dataset, steps=validation_steps)
print(f"\nTest Loss: {results[0]:.4f} | Test Accuracy: {results[1]:.4f}")


Encoding texts: 100%|██████████| 10/10 [00:00<00:00, 2649.76it/s]
TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.


TypeError: 'builtins.safe_open' object is not iterable